<h1 style="text-align:center;font-size:40px;">Tools 1 Final Project</h1>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<p style="text-align:center;">Anthony Molieri<br>
Leo Walker</p>
<br>
<br>
<br>
<br>

# Dataset and Motivation

The data used for these analyses were scraped from Indeed and Glassdoor. The data from Indeed contains job descriptions for open Data Scientist positions. The Glassdoor data includes employee generated ratings on benefits and perks for many of the companies present in the Indeed dataset. The datasets were then combined in order to give an idea of how employees rank the benefits at these companies.

*The Indeed data was scraped over ... pulling job descriptions for Data Scientists in 25 cities throughout the US. **Leo** Add Correct Details, here about the Indded data and scraper*

The Glassdoor scraper was designed to pull company names from the the job description dataset. It then looped over the list of employers, searched, and scraped ratings from the benefits page for companies that were found. This data was then added to it's own table in the same database. The scraper was designed to be run multiple times, only searching for benefits ratings that were not already present in the database. 

# Task Definition and Research Question
 
The purpose of this project was to help give a better idea of how employees at companies with open positions rate their benefits. This can help give insight into how desirable a company is as an employer.

# Literature Review

*Add -- I'm not really sure what to add here. Maybe something about turnover in data science? Companies not having great benefits?*

# Data Cleaning

In [7]:
# imports
import numpy as np
import pandas as pd
import sqlite3
import re

In [8]:
# Initialize both tables
cnct = sqlite3.connect('Data/jobs')

indeed_df = pd.read_sql_query('SELECT * FROM indeed_jobs;', cnct)
indeed_df.drop('est salary', axis=1, inplace=True)

glassdoor_df = pd.read_sql_query('SELECT companies, type, rating, count_of_ratings FROM ratings;', cnct)

## Cleaning Indeed Data

In [9]:
# While we were able to get the estimated salary from the job scraping we can see if there was any job salaries within the job description
def salary_extract(row):
    if row['est_salary']!="No Estimated Salary":
        return row['est_salary']
    else:
        pattern = r"\$([0-9]{5,6}|[0-9]{2,3},[0-9]{3})"
        found = re.findall(pattern, row['job_desc'])
        if len(found)==1:
            return f'{found[0]} a year'
        elif len(found)>1:
            return f'{found[0]} - {found[1]} a year'
        else:
            return None

indeed_df['salary_extract'] = indeed_df.apply(salary_extract, axis=1)

In [10]:
# While we have the job title we need to categorize each job
def set_job_category(job_title):
    regex = re.compile('[^a-zA-Z]')
    clean_title = regex.sub('', job_title)
    clean_title = clean_title.lower()
    if all(x in clean_title for x in ["data", "scien"]):
        return "Data Scientist"
    elif all(x in clean_title for x in ["business", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["bi", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["data", "analy"]):
        return "Data Analyst"
    elif all(x in clean_title for x in ["machine", "learning"]):
        return "Machine Learning Engineer"
    elif all(x in clean_title for x in ["data", "engineer"]):
        return "Data Engineer"
    elif all(x in clean_title for x in ["software", "engin"]):
        return "Software Engineer"
    elif all(x in clean_title for x in ["architect"]):
        return "Data Architect"
    else:
        return "Other"

indeed_df['job_category'] = indeed_df.loc[:, 'job_title'].apply(set_job_category)

In [11]:
# Lets get the state from the company location so we can generalize them
indeed_df.loc[:,'state'] = indeed_df.loc[:, 'company_location'].str.extract(r', ([A-Z]{2})')

In [14]:
indeed_df.salary_extract

0        None
1        None
2        None
3        None
4        None
         ... 
17181    None
17182    None
17183    None
17184    None
17185    None
Name: salary_extract, Length: 17186, dtype: object

In [49]:
# We have the estimated salary but we need to parse out the min/low and max/high of the salary range so we can determine the average

def salary_low(row):
    if row ==None:
        return None
    salary = str(row).split('-')
    pattern = r'([0-9]+\.*,*[0-9]+)'
    low = float(''.join(re.findall(pattern, salary[0])).replace(',',''))
    return low

def salary_high(row):
    if row ==None:
        return None
    salary = str(row).split('-')
    pattern = r'([0-9]+\.*,*[0-9]+)'
    if len(salary)>1:
        high = float(''.join(re.findall(pattern, salary[1])).replace(',',''))
    else:
        high = float(''.join(re.findall(pattern, salary[0])).replace(',',''))
    return high

indeed_df.loc[:,['salary_low']] = indeed_df.loc[:,'salary_extract'].apply(salary_low)
indeed_df.loc[:,['salary_high']] = indeed_df.loc[:,'salary_extract'].apply(salary_high)

In [45]:
# We noticed that the pay rate varies between annual, month, week, day, or hour. 
def pay_rate(row):
    if row ==None:
        return None
    regex = re.compile('[.[0-9]]')
    clean_salary = regex.sub('', row)
    clean_salary = clean_salary.lower()
    if any(x in clean_salary for x in ["year"]):
        return "annual"
    elif any(x in clean_salary for x in ["month"]):
        return "month"
    elif any(x in clean_salary for x in ["week"]):
        return "week"
    elif any(x in clean_salary for x in ["day"]):
        return "day"
    elif any(x in clean_salary for x in ["hour"]):
        return "hour"
    else:
        return "Other"

indeed_df['pay_rate'] = indeed_df.loc[:, 'salary_extract'].apply(pay_rate)

In [46]:
def calc_salary(row):
    # print(row)
    pay_rate=row['pay_rate']
    if (pay_rate == 'Other') or (pay_rate==None):
        return None

    rate_change= {'annual':1, 'month':12, 'week':52, 'day':260, 'hour':2080}
    est_annual_salary = ((row['salary_low']+row['salary_high'])/2)*rate_change[pay_rate]
    return est_annual_salary
indeed_df['est_annual_salary'] = indeed_df.loc[:,['salary_low','salary_high','pay_rate']].apply(calc_salary, axis=1)

In [48]:
indeed_df[indeed_df['est_salary']!='No Estimated Salary'].sample(4)

,index,job_title,company_name,company_location,job_href,job_desc,est_salary,retrieve_date,salary_extract,job_category,state,salary_low,salary_high,pay_rate,est_annual_salary
2200,0,Data Engineer,Seattle Kraken,"Seattle, WA 98125 (Northgate area)",https://www.indeed.com/company/Kraken-Communit...,Our Purpose:We are building a championship org...,"$85,000 - $100,000 a year",2022-02-12,"$85,000 - $100,000 a year",Data Engineer,WA,85000.00,100000.00,annual,92500.0
3688,10,Business Analyst - 4187475,Accenture,"Boston, MA",https://www.indeed.com/rc/clk?jk=9199ca6833f96...,Accenture Flex offers you the flexibility of l...,$28.76 - $54.66 an hour,2022-02-19,$28.76 - $54.66 an hour,Business Intelligence Analyst,MA,28.76,54.66,hour,86756.8
15068,2,Engineer 2 - Fraud Team,Nordstrom Inc,"Seattle, WA",https://www.indeed.com/rc/clk?jk=61f0cd9bac2a4...,Job Description An Engineer 2 is part...,"$99,000 - $153,500 a year",2022-04-10,"$99,000 - $153,500 a year",Other,WA,99000.00,153500.00,annual,126250.0
12557,5,Senior Data Scientist,CyberCoders,"Los Angeles, CA 90034+1 location",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Senior Data Scientist If you are a ...,"$150,000 - $170,000 a year",2022-04-03,"$150,000 - $170,000 a year",Data Scientist,CA,150000.00,170000.00,annual,160000.0


# Visualizations